In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.optimizers as opt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, MaxPooling1D, BatchNormalization, Activation, Layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.initializers import  RandomNormal, Constant
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Input

import lightgbm as lgb
from lightgbm import LGBMClassifier, plot_metric
import xgboost as xgb

import numpy as np
# import pandas as pd

In [2]:
path = "./img_sub/cat"
dirlist = os.listdir(path)
catPath = [os.path.join(path, dirname) for dirname in dirlist]

total_img_num = sum(map(len,map(os.listdir, catPath)))
print('all kinds of cats: ', dirlist)
for i in range(len(catPath)):
    print(dirlist[i], ': ', len(os.listdir(catPath[i])))
print('total cat imgs: ', total_img_num)

all kinds of cats:  ['bljfm', 'bom', 'dlm', 'fksm', 'jjc', 'jm', 'lm', 'md', 'mgbwm', 'mmm', 'mym', 'nnm', 'xmly', 'yjc', 'zem']
bljfm :  258
bom :  257
dlm :  222
fksm :  222
jjc :  246
jm :  232
lm :  270
md :  258
mgbwm :  235
mmm :  212
mym :  222
nnm :  200
xmly :  204
yjc :  234
zem :  235
total cat imgs:  3507


In [3]:
batch_size = 10
epochs = 5
IMG_HEIGHT = 116
IMG_WIDTH = 116
split = 0.2
classNum = len(dirlist)
total_val = total_img_num * split
total_train = total_img_num * (1 - split)

In [4]:
# Generator

train_datagen = ImageDataGenerator(
    rescale=1./255,rotation_range=40, # Angle, 0-180
    width_shift_range=0.2, # horizontal shifting
    height_shift_range=0.2, # vertical shifting
    shear_range=0.2, # Shearing
    zoom_range=0.2, # Zooming
    horizontal_flip=True, # Flipping
    validation_split=split
)

test_datagen = ImageDataGenerator(
    rescale=1./255,rotation_range=40, # Angle, 0-180
    width_shift_range=0.2, # horizontal shifting
    height_shift_range=0.2, # vertical shifting
    shear_range=0.2, # Shearing
    zoom_range=0.2, # Zooming
    horizontal_flip=True, # Flipping
)

In [5]:
# Load images from the disk, applies rescaling, and resizes the images

train_generator = train_datagen.flow_from_directory(
    directory=path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
) # set as training data

validation_generator = train_datagen.flow_from_directory(
    directory=path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
) # set as validation data

print("Train set shape: ", train_generator.classes.shape)
print("Validation/Test set shape: ", validation_generator.classes.shape)

Found 2811 images belonging to 15 classes.
Found 696 images belonging to 15 classes.
Train set shape:  (2811,)
Validation/Test set shape:  (696,)


In [6]:
model = load_model('./multiclass/f_3355_60.180.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  

In [8]:
a = model

In [9]:
a.fit_generator(
        train_generator,
        steps_per_epoch = total_train // batch_size,
        epochs = epochs,
        validation_data = validation_generator,
        validation_steps = total_val // batch_size     
    )

Epoch 1/5
280/280 [==============================] - 277s 988ms/step - loss: 2.5807 - acc: 0.1627 - val_loss: 2.3202 - val_acc: 0.2371
Epoch 2/5
280/280 [==============================] - 311s 1s/step - loss: 2.2438 - acc: 0.2700 - val_loss: 2.1550 - val_acc: 0.2888
Epoch 3/5
165/280 [================>.............] - ETA: 1:54 - loss: 2.1117 - acc: 0.3254

In [0]:
dense1_layer_model = Model(inputs=a.input,
                           outputs=a.get_layer('act5').output)

dense1_output = dense1_layer_model.predict(train_generator)
dense1_val = dense1_layer_model.predict(validation_generator)

In [0]:
x_train = dense1_output
y_train = train_generator.classes
x_test = dense1_val
y_test = validation_generator.classes

print(x_test)
print(y_test)

In [0]:
xb = xgb.XGBClassifier()
xb.fit(x_train, y_train)

In [0]:
print(xb.score(x_train,y_train))
print(xb.score(x_test,y_test))

In [ ]:
params = {"objective" : "softmax",
          "num_class" : 15,
          "num_leaves" : 50,
          "max_depth": -1,
          "learning_rate" : 0.01,
          "bagging_fraction" : 0.9,  # subsample
          "feature_fraction" : 0.9,  # colsample_bytree
          "bagging_freq" : 5,        # subsample_freq
          "bagging_seed" : 2018,
          "verbosity" : -1,
          "metric" : "softmax"
}

classifier_params = {
    'boosting_type' : 'gbdt',
    'num_leaves' : 31,
    'max_depth' : -1,
    'learning_rate' : 0.1,
    'n_estimators' : 100,
    'subsample_for_bin' : 200000,
    'objective' : 'softmax',
    'class_weight' : None,
    'min_split_gain' : 0.0,
    'min_child_weight' : 0.001,
    'min_child_samples' : 20,
    'subsample' : 1.0,
    'subsample_freq' : 0,
    'colsample_bytree' : 1.0,
    'reg_alpha' : 0.0,
    'reg_lambda' : 0.0,
    'random_state' : '10927',
    'n_jobs' : -1,
    'silent' : True,
    'importance_type' : 'split'
}

In [ ]:
dataset = lgb.Dataset(x_train, label = y_train)
dataset.construct()
vali = dataset.create_valid(x_test, label = y_test)
vali.construct()
print(dataset.num_data())
print(vali.num_data())

In [ ]:
# eval_hist = lgb.cv(params, train_set = dataset)
# trained_model = lgb.train(params, train_set = dataset)
trained_model = lgb.train(params, train_set = dataset)

In [ ]:
classifier = LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=20,
    max_depth=50,
    learning_rate=0.01,
    n_estimators=30,
    subsample_for_bin=10,
    objective='softmax',
    class_weight=None,
    min_split_gain=0.0,
    min_child_weight=0.001,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state='10927',
    n_jobs=-1,
    silent=True,
    importance_type='split'
)

In [ ]:
classifier.fit(x_train, y_train)

In [ ]:
result = classifier.predict(x_test)

In [ ]:
corr = 0
for i in range(len(result)):
    if result[i] == y_test[i]:
        corr += 1
print(corr*1.0/len(result))
print(y_test)
print(result)